## Creating a mesh

We can also use the Splatter wrapper class to take an existing nerfstudio model and create a mesh!
1. **mesh:** creates a mesh via TSDF fusion

2. **query_mesh:** uses the trained model to query the mesh and returns a similarity map

3. **plot_mesh:** enables plotting of mesh features



In [1]:
import os, sys
from pathlib import Path
from ns_extension.wrapper import Splatter, SplatterConfig

import pyvista as pv
pv.start_xvfb()

# import vtkmodules.all as vtk
# render_window = vtk.vtkRenderWindow()
# render_window.SetOffScreenRendering(1)


/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/pyvista/plotting/utilities/xvfb.py:48: PyVistaDeprecationWarning: This function is deprecated and will be removed in future version of PyVista. Use vtk-osmesa instead.
  warnings.warn(


Set paths to the file for running splats

In [2]:
base_dir = Path('/workspace/fieldwork-data/')
session_dir = base_dir / "rats/2024-07-11/SplatsSD"

# Make the configuration 
splatter_config = SplatterConfig(
    file_path=session_dir / "C0119.MP4",
    method='rade-features',
    frame_proportion=0.25, # Use 25% of the frames within the video (or default to minimum 300 frames)
)

# Initialize the Splatter class
splatter = Splatter(splatter_config)

# Call these to populate the splatter with paths (probably a better way to do this --> maybe save out config)
splatter.preprocess()
splatter.extract_features()

transforms.json already exists at /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/preproc/transforms.json
To rerun preprocessing, set overwrite=True
Output already exists for rade-features
To rerun feature extraction, set overwrite=True


### Create a mesh

We can create a mesh by calling the ```mesh()``` method. Under the hood, this runs TSDF fusion creating an integrated volume. 

In [3]:
splatter.mesh(
    depth_name="median_depth",
)


Available runs:
[0] 2025-07-11_171420


### Plot the mesh!

We can use the splatter function ```plot_mesh``` to visualize given attributes of the mesh. The inherent attributes are RGB and Normals

In [15]:
splatter.plot_mesh(attribute="RGB")

Number of points: 277426
Number of cells: 494857
Bounds: BoundsTuple(x_min=-1.4198578596115112, x_max=0.7450000047683716, y_min=-0.125, y_max=3.0850000381469727, z_min=-1.459478735923767, z_max=1.81288480758667)


Widget(value='<iframe src="http://localhost:32829/index.html?ui=P_0x700634e8a770_3&reconnect=auto" class="pyvi…

### Using semantic queries 

The mesh contains semantic features which we can query via positive and negative prompts. The goal of this is to find points that are more similar to the positive prompts compared to the negative prompts

In [16]:
similarity = splatter.query_mesh(
    positive_queries=["tree"],
    negative_queries=["ground", "leaves"],
)

Loading model from /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/config.yml
[Taichi] version 1.7.3, llvm 15.0.4, commit 5ec301be, linux, python 3.10.18


[I 07/21/25 16:04:48.915 34474] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[16:04:52] Auto image downscale factor of 2                                                 ]8;id=321733;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=53397;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


KeyboardInterrupt: 

Plot similarity maps

In [ ]:
splatter.plot_mesh(attribute=similarity, rgb=False)

Number of points: 277426
Number of cells: 494857
Bounds: BoundsTuple(x_min=-1.4198578596115112, x_max=0.7450000047683716, y_min=-0.125, y_max=3.0850000381469727, z_min=-1.459478735923767, z_max=1.81288480758667)


Widget(value='<iframe src="http://localhost:36567/index.html?ui=P_0x76a4efb16aa0_3&reconnect=auto" class="pyvi…

### Lets clean up the mesh

In [ ]:
import trimesh
import pymeshfix
import numpy as np
from scipy.spatial import cKDTree

def clean_repair_mesh(mesh_path: str) -> trimesh.Trimesh:
    """
    Load a mesh, extract the largest component, repair it conservatively,
    and map vertex properties (color, normals) back to the repaired mesh.
    """
    # Load mesh
    # mesh = trimesh.load(mesh_path, process=False)
    mesh = pv.read(mesh_path)
    print(f"[INFO] Loaded mesh with {mesh.n_points} vertices and {mesh.n_cells} faces")

    # # Get largest connected component
    # components = mesh.split(only_watertight=False)
    # largest = max(components, key=lambda c: len(c.vertices))
    # print(f"[INFO] Largest component: {len(largest.vertices)} vertices, {len(largest.faces)} faces")

    # Conservatively repair mesh (no component joining or small-part removal)
    fixer = pymeshfix.MeshFix(mesh)
    fixer.repair(verbose=True, joincomp=False, remove_smallest_components=False)

    # # Reconstruct mesh
    # repaired_vertices = fixer.v
    # repaired_faces = fixer.f
    # repaired_mesh = trimesh.Trimesh(vertices=repaired_vertices, faces=repaired_faces, process=False)

    # # Map vertex properties using nearest neighbors
    # tree = cKDTree(largest.vertices)
    # _, indices = tree.query(repaired_vertices)

    # if hasattr(largest, 'vertex_colors') and len(largest.vertex_colors) > 0:
    #     repaired_mesh.visual.vertex_colors = largest.vertex_colors[indices]

    # if hasattr(largest, 'vertex_normals') and len(largest.vertex_normals) > 0:
    #     repaired_mesh.vertex_normals = largest.vertex_normals[indices]

    # print("[INFO] Repaired mesh constructed and attributes mapped.")
    return fixer #, largest

In [ ]:
import numpy as np
import pyvista as pv
import pymeshfix
from scipy.spatial import cKDTree

def repair_mesh(mesh_path: str, verbose=True) -> pv.PolyData:
    """
    Repair a PyVista mesh using pymeshfix while retaining vertex properties.
    
    Parameters:
        mesh (pv.PolyData): Input PyVista mesh to repair.
        verbose (bool): Whether to print pymeshfix verbose output.
    
    Returns:
        pv.PolyData: Repaired mesh with vertex properties mapped from original.
    """
    
    mesh = pv.read(mesh_path)

    if verbose:
        print(f"[INFO] Input mesh: {mesh.n_points} vertices, {mesh.n_faces} faces")

    # Extract vertices and faces from PyVista mesh
    vertices = mesh.points
    # PyVista faces format: [N, v0, v1, v2, N, v0, v1, v2,...]
    faces = mesh.faces.reshape((-1, 4))[:, 1:4]

    # Repair mesh with pymeshfix
    mf = pymeshfix.MeshFix(vertices, faces)
    mf.repair(verbose=verbose, joincomp=False, remove_smallest_components=False)

    repaired_vertices = mf.v
    repaired_faces = mf.f

    if verbose:
        print(f"[INFO] Repaired mesh: {len(repaired_vertices)} vertices, {len(repaired_faces)} faces")

    # Build PyVista faces array format: [3, i0, i1, i2, 3, ...]
    flat_faces = np.hstack(
        np.column_stack((np.full(len(repaired_faces), 3, dtype=np.int64), repaired_faces))
    )

    repaired_mesh = pv.PolyData(repaired_vertices, flat_faces)

    # Map original vertex properties via nearest neighbor
    tree = cKDTree(vertices)
    distances, indices = tree.query(repaired_vertices, k=1)

    for name in mesh.point_data.keys():
        original_array = mesh.point_data[name]
        if len(original_array) == mesh.n_points:
            mapped_array = original_array[indices]
            repaired_mesh.point_data[name] = mapped_array
            if verbose:
                print(f"[INFO] Mapped vertex property '{name}'")

    return repaired_mesh

### Fix mesh multistep

In [ ]:
import pyvista as pv
import pymeshfix
import numpy as np

mesh = pv.read(splatter.config['mesh_info']['mesh'])


# Compute connectivity (default to cell connectivity)
connectivity = mesh.connectivity()
region_ids = connectivity.cell_data['RegionId'] # The 'RegionId' array labels each connected component
unique_ids, counts = np.unique(region_ids, return_counts=True) # Count the number of cells in each region
largest_region_id = unique_ids[np.argmax(counts)] # Find the region id with the max number of cells

# Extract cells belonging to largest connected component
lcc = connectivity.extract_cells(region_ids == largest_region_id)

print(f"Largest connected component has {lcc.n_points} points and {lcc.n_cells} cells")

mesh = lcc.extract_surface() # Extract surface as PolyData



# Extract surface as PolyData

# max_hole_size = 1e6 # tune this based on hole size (units = mesh units)
# mesh = mesh.fill_holes(max_hole_size)

Largest connected component has 164314 points and 302746 cells


In [ ]:
def clean_mesh(mesh_path: str):
    mesh = pv.read(mesh_path)

    # Compute connectivity (default to cell connectivity)
    connectivity = mesh.connectivity()
    region_ids = connectivity.cell_data['RegionId'] # The 'RegionId' array labels each connected component
    unique_ids, counts = np.unique(region_ids, return_counts=True) # Count the number of cells in each region
    largest_region_id = unique_ids[np.argmax(counts)] # Find the region id with the max number of cells

    # Extract cells belonging to largest connected component
    lcc = connectivity.extract_cells(region_ids == largest_region_id)

    print(f"Largest connected component has {lcc.n_points} points and {lcc.n_cells} cells")

    mesh = lcc.extract_surface() # Extract surface as PolyData

    return mesh

In [11]:
# sphinx_gallery_thumbnail_number = 1
import numpy as np
import pyvista as pv

from pymeshfix import MeshFix
from pymeshfix._meshfix import PyTMesh
from pymeshfix.examples import planar_mesh

In [12]:

meshfix = MeshFix(mesh)
holes = meshfix.extract_holes()

In [ ]:
meshfix.repair(verbose=True)

In [14]:
# Render the mesh and outline the holes
plotter = pv.Plotter()
plotter.add_mesh(mesh, color=True)
plotter.add_mesh(holes, color="r", line_width=5)
plotter.enable_eye_dome_lighting()  # helps depth perception
_ = plotter.show()

Widget(value='<iframe src="http://localhost:45081/index.html?ui=P_0x7e0525f82770_4&reconnect=auto" class="pyvi…

In [9]:
edges = mesh.extract_feature_edges(
    boundary_edges=True,
    feature_edges=False,
    manifold_edges=False,
    non_manifold_edges=False,
)

print(f"Number of boundary edges: {edges.n_lines}")
edges.plot()

Number of boundary edges: 31312


Widget(value='<iframe src="http://localhost:45855/index.html?ui=P_0x7c3948f0bbb0_3&reconnect=auto" class="pyvi…

In [16]:
import pyvista as pv
import numpy as np
import pymeshfix
from scipy.spatial import Delaunay, cKDTree
from tqdm import tqdm  # Optional, for progress bars

def fill_small_holes_selectively(mesh: pv.PolyData,
                                hole_perimeter_threshold=1.0,
                                max_edge_length=0.2,
                                verbose=True) -> pv.PolyData:
    """
    Fill only holes smaller than hole_perimeter_threshold and remove suspicious triangles.
    
    Parameters:
        mesh (pv.PolyData): Input mesh (must be PolyData)
        hole_perimeter_threshold (float): Max perimeter length of holes to fill
        max_edge_length (float): Max allowed edge length after repair to keep triangles
        verbose (bool): Print progress info
    
    Returns:
        pv.PolyData: Hole-filled and cleaned mesh
    """
    # if verbose:
    print(f"[INFO] Starting with mesh: {mesh.n_points} points, {mesh.n_faces} faces")

    # Step 1: Extract boundary edges (hole loops)
    boundary_edges = mesh.extract_feature_edges(
        boundary_edges=True,
        feature_edges=False,
        manifold_edges=False,
        non_manifold_edges=False,
    )
    
    if boundary_edges.n_lines == 0:
        # if verbose: 
        print("[INFO] No boundary edges (holes) found. Returning original mesh.")
        return mesh.copy()
    
    # if verbose:
    print(f"[INFO] Found {boundary_edges.n_lines} hole loops")

    # Step 2: Separate boundary edges into connected loops
    connectivity = boundary_edges.connectivity()
    region_ids = connectivity.point_data['RegionId']

    filled_mesh = mesh.copy()

    def order_loop_points_fast(pts):
        pts = np.asarray(pts)
        N = len(pts)
        tree = cKDTree(pts)
        ordered = [0]
        used = set(ordered)

        while len(ordered) < N:
            last_idx = ordered[-1]
            dists, idxs = tree.query(pts[last_idx], k=10)
            for i in idxs:
                if i not in used:
                    ordered.append(i)
                    used.add(i)
                    break
            else:
                # No unused neighbor found - break to avoid infinite loop
                break
        return pts[ordered]

    unique_region_ids = np.unique(region_ids)
    iterator = tqdm(unique_region_ids, desc="Filling holes")

    for region_id in iterator:
        hole_loop = connectivity.extract_points(region_ids == region_id, adjacent_cells=True)
        loop_pts = hole_loop.points
        ordered_pts = order_loop_points_fast(loop_pts)

        perimeter = np.sum(np.linalg.norm(np.diff(np.vstack([ordered_pts, ordered_pts[0]]), axis=0), axis=1))
        # if verbose:
        #     print(f" - Hole region {region_id} perimeter: {perimeter:.3f}")

        if perimeter < hole_perimeter_threshold:
            # if verbose:
            #     print("   Filling hole...")
            # Project points to best-fit plane
            centroid = ordered_pts.mean(axis=0)
            pts_centered = ordered_pts - centroid
            _, _, vh = np.linalg.svd(pts_centered)
            proj_pts = pts_centered @ vh[:2].T

            tri = Delaunay(proj_pts)
            hole_faces = tri.simplices

            # Add new vertices and faces
            vertices = filled_mesh.points
            faces = filled_mesh.faces.reshape((-1, 4))[:, 1:4]

            hole_vertex_indices = np.arange(len(vertices), len(vertices) + len(ordered_pts))
            new_vertices = np.vstack([vertices, ordered_pts])
            new_faces = hole_faces + len(vertices)

            combined_faces = np.vstack([faces, new_faces])

            flat_faces = np.hstack(
                np.column_stack((np.full(len(combined_faces), 3, dtype=int), combined_faces))
            )
            filled_mesh = pv.PolyData(new_vertices, flat_faces)
        else:
            # if verbose:
            print("   Hole too large, skipping filling.")

    # if verbose:
    print("[INFO] Running pymeshfix repair...")

    vertices = filled_mesh.points
    faces = filled_mesh.faces.reshape((-1, 4))[:, 1:4]

    mf = pymeshfix.MeshFix(vertices, faces)
    mf.repair(verbose=verbose, joincomp=False, remove_smallest_components=False)

    repaired_vertices = mf.v
    repaired_faces = mf.f

    flat_faces = np.hstack(
        np.column_stack((np.full(len(repaired_faces), 3, dtype=int), repaired_faces))
    )

    repaired_mesh = pv.PolyData(repaired_vertices, flat_faces)

    if verbose:
        print(f"[INFO] Removing triangles with edges longer than {max_edge_length}...")

    triangles = repaired_mesh.faces.reshape((-1, 4))[:, 1:4]
    points = repaired_mesh.points

    tri_verts = points[triangles]
    edge_vecs = np.roll(tri_verts, -1, axis=1) - tri_verts
    edge_lengths = np.linalg.norm(edge_vecs, axis=2)
    mask = np.all(edge_lengths <= max_edge_length, axis=1)

    filtered_triangles = triangles[mask]

    flat_faces = np.hstack(
        np.column_stack((np.full(len(filtered_triangles), 3, dtype=int), filtered_triangles))
    )

    final_mesh = pv.PolyData(points, flat_faces)

    if verbose:
        print(f"[INFO] Final mesh: {final_mesh.n_points} points, {final_mesh.n_faces} faces")

    return final_mesh


In [17]:
filled_mesh = fill_small_holes_selectively(mesh, verbose=False)

[INFO] Starting with mesh: 164314 points, 302746 faces
[INFO] Found 31312 hole loops


Filling holes:   0%|          | 3/1249 [00:00<04:30,  4.60it/s]

   Hole too large, skipping filling.


Filling holes:   1%|          | 9/1249 [00:01<04:11,  4.93it/s]

   Hole too large, skipping filling.


Filling holes:   1%|▏         | 17/1249 [00:03<04:23,  4.68it/s]

   Hole too large, skipping filling.


Filling holes: 100%|██████████| 1249/1249 [04:06<00:00,  5.08it/s]


[INFO] Running pymeshfix repair...


In [4]:
import pyvista as pv
import pymeshfix
import numpy as np

mesh = pv.read(splatter.config['mesh_info']['mesh'])


# Compute connectivity (default to cell connectivity)
connectivity = mesh.connectivity()

# The 'RegionId' array labels each connected component
region_ids = connectivity.cell_data['RegionId']

# Count the number of cells in each region
unique_ids, counts = np.unique(region_ids, return_counts=True)

# Find the region id with the max number of cells
largest_region_id = unique_ids[np.argmax(counts)]

# Extract cells belonging to largest connected component
lcc = connectivity.extract_cells(region_ids == largest_region_id)

print(f"Largest connected component has {lcc.n_points} points and {lcc.n_cells} cells")

# Extract surface as PolyData
mesh = lcc.extract_surface()

# orig_mesh.plot_boundaries()

meshfix = pymeshfix.MeshFix(mesh)
# holes = meshfix.extract_holes()

# mfix = pymeshfix.PyTMesh(False)  # False removes extra verbose output
# mfix.load_file(splatter.config['mesh_info']['mesh'])


# # Fills all the holes having at at most 'nbe' boundary edges. If
# # 'refine' is true, adds inner vertices to reproduce the sampling
# # density of the surroundings. Returns number of holes patched.  If
# # 'nbe' is 0 (default), all the holes are patched.
# meshfix.fill_small_boundaries(nbe=100, refine=True)


Largest connected component has 164314 points and 302746 cells


In [ ]:
meshfix.repair()

In [30]:
# Render the mesh and outline the holes
plotter = pv.Plotter()
plotter.add_mesh(orig_mesh, color=True)
plotter.add_mesh(holes, color="r", line_width=5)
plotter.enable_eye_dome_lighting()  # helps depth perception
_ = plotter.show()

Widget(value='<iframe src="http://localhost:40585/index.html?ui=P_0x700365f90160_7&reconnect=auto" class="pyvi…

In [21]:
import pyvista as pv

# mesh = pv.read(splatter.config['mesh_info']['mesh'])

plotter = pv.Plotter()
plotter.add_mesh(filled_mesh) #, scalars='RGB', rgb=True)
plotter.show()

Widget(value='<iframe src="http://localhost:40585/index.html?ui=P_0x700368e76ef0_3&reconnect=auto" class="pyvi…

In [49]:
import pyvista as pv

# mesh = pv.read(splatter.config['mesh_info']['mesh'])

mesh = clean_repair_mesh(splatter.config['mesh_info']['mesh'])
# mesh.fill_holes()

# mesh.show()

[INFO] Loaded mesh with 277426 vertices and 494857 faces
INFO- Loaded 277426 vertices and 494857 faces.
Patching holes...
Patched 2522 holes
Fixing degeneracies and intersections

100% done done 
INFO- ********* ITERATION 0 *********
INFO- Removing degeneracies...
INFO- Removing self-intersections...

99 % done   
INFO- 43012 intersecting triangles have been selected.

99 % done   
INFO- 4658 intersecting triangles have been selected.

99 % done   
INFO- 1963 intersecting triangles have been selected.
INFO- ********* ITERATION 1 *********
INFO- Removing degeneracies...
INFO- Removing self-intersections...

99 % done   
INFO- 1393 intersecting triangles have been selected.

99 % done   
INFO- 2265 intersecting triangles have been selected.

14 % done   

99 % done   
INFO- 762 intersecting triangles have been selected.
INFO- ********* ITERATION 2 *********
INFO- Removing degeneracies...
INFO- Removing self-intersections...

99 % done   
INFO- 1145 intersecting triangles have been selected.

99 % done   
INFO- 575 intersecting triangles have been selected.

99 % done   
INFO- 443 intersecting triangles have been selected.
INFO- ********* ITERATION 3 *********
INFO- Removing degeneracies...
INFO- Removing self-intersections...

99 % done   
INFO- 289 intersecting triangles have been selected.

99 % done   
INFO- 308 intersecting triangles have been selected.

99 % done   
INFO- 455 intersecting triangles have been selected.
INFO- ********* ITERATION 4 *********
INFO- Removing degeneracies...
INFO- Removing self-intersections...

99 % done   
INFO- 272 intersecting triangles have been selected.

99 % done   
INFO- 534 intersecting triangles have been selected.

98 % done   
INFO- 324 intersecting triangles have been selected.
INFO- ******

In [61]:
import open3d as o3d
import numpy as np

def clean_mesh(mesh, min_cluster_size=1000):

    # Method 1: Connected component analysis
    triangle_clusters, cluster_n_triangles, cluster_area = (
        mesh.cluster_connected_triangles())
    triangle_clusters = np.asarray(triangle_clusters)
    cluster_n_triangles = np.asarray(cluster_n_triangles)
    cluster_area = np.asarray(cluster_area)

    print(f"Found {len(cluster_n_triangles)} connected components")
    print(f"Component sizes: {cluster_n_triangles}")

    # Remove small components
    large_cluster_ids = np.where(cluster_n_triangles > min_cluster_size)[0]

    # Create mask for triangles to keep
    triangles_to_keep = np.isin(triangle_clusters, large_cluster_ids)

    # Remove small components
    mesh.remove_triangles_by_mask(~triangles_to_keep)
    mesh.remove_unreferenced_vertices()
    
    # Additional cleanup
    mesh.remove_degenerate_triangles()
    mesh.remove_duplicated_triangles()
    mesh.remove_duplicated_vertices()
    mesh.remove_non_manifold_edges()
    
    return mesh

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [64]:
import open3d as o3d

mesh_path = splatter.config['mesh_info']['mesh']
mesh = o3d.io.read_triangle_mesh(mesh_path)

mesh = clean_mesh(mesh, min_cluster_size=10000)

Found 50 connected components
Component sizes: [302746   5573   6016   5463  11909  18670   1676   1780   2339   2017
   2191   4042   9860   3720   3050   4876   2490   5448  10420   2969
   2528   2207   2877   2631   2537   2267   2761   2940   7396   2348
   2376   2689   1839   5457   6696   2601   3542   1938   3534   2402
   4030   1825   2396   2108   2142   1952   3033   2882   1703   1965]


### Poisson repair